## Build base Docker image (if not already built) 

In [4]:
%%bash
docker build -t mlflow-docker-example -f bike-rental-regression/Dockerfile . 

#1 [internal] load build definition from Dockerfile
#1 sha256:d312112363cadc9fedb2358b4847d4e1b465cf403776d6fe13117c6a28284241
#1 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 sha256:d312112363cadc9fedb2358b4847d4e1b465cf403776d6fe13117c6a28284241
#1 transferring dockerfile: 37B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:00da033bf2f3f75752733f4d73f495e88806c5fa7991ae5509f1cd8cf51451db
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/continuumio/miniconda3:4.10.3
#3 sha256:f692b03fa4ad9171171f333a701a320801f86a5a153c52ecb1003a5c4c2ae3ce
<3>init: (7582) ERROR: UtilConnectToInteropServer:307: connect failed 2
<3>init: (7589) ERROR: UtilConnectToInteropServer:307: connect failed 2
<3>init: (7590) ERROR: UtilConnectToInteropServer:307: connect failed 2
#3 ERROR: rpc error: code = Unknown desc = error getting credentials - err: exit status 1, out: ``
------
 > [internal] load metadata for docker.io/continuumio/min

CalledProcessError: Command 'b'docker build -t mlflow-docker-example -f bike-rental-regression/Dockerfile . \n'' returned non-zero exit status 1.

## Train a model 

In [2]:
%%bash
mlflow run bike-rental-regression -P train_year=2011 -P train_month=1 -P test_year=2011 -P test_month=3 -P dataset_path=/data/train_bike.csv

2021/11/17 14:35:14 INFO mlflow.projects.docker: === Building docker image bike-rental-regression:548b989 ===
2021/11/17 14:35:20 INFO mlflow.projects.utils: === Created directory /tmp/tmp774uji4r for downloading remote URIs passed to arguments of type 'path' ===
2021/11/17 14:35:20 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/andrea/demo-ATSS/mlruns:/mlflow/tmp/mlruns -v /home/andrea/demo-ATSS/mlruns/0/d20fd3db6b3f4d02969b704b993b02a8/artifacts:/home/andrea/demo-ATSS/mlruns/0/d20fd3db6b3f4d02969b704b993b02a8/artifacts -v /home/andrea/demo-ATSS/data:/data -e MLFLOW_RUN_ID=d20fd3db6b3f4d02969b704b993b02a8 -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 bike-rental-regression:548b989 python train.py 2011 1 2011 3 /data/train_bike.csv' in run with ID 'd20fd3db6b3f4d02969b704b993b02a8' === 
/opt/conda/lib/python3.9/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Inte

## Package the model in a Docker image and register it in the local registry

In [3]:
%%bash
mlflow models build-docker --model-uri runs:/c258b637e0a64ffe979248f475d4c94b/model --name "my-image-name"

/tmp/tmpdowwsj7b/
/tmp/tmpdowwsj7b/model_dir
/tmp/tmpdowwsj7b/model_dir/model
/tmp/tmpdowwsj7b/model_dir/model/model.pkl
/tmp/tmpdowwsj7b/model_dir/model/conda.yaml
/tmp/tmpdowwsj7b/model_dir/model/MLmodel
/tmp/tmpdowwsj7b/model_dir/model/requirements.txt
/tmp/tmpdowwsj7b/model_dir/model/input_example.json
/tmp/tmpdowwsj7b/Dockerfile


2021/11/17 14:35:56 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2021/11/17 14:35:56 INFO mlflow.models.docker_utils: Building docker image with name my-image-name
#1 [internal] load build definition from Dockerfile
#1 sha256:de9bcfd240c615dc2b3fb625fc127a4cf7c65691cd9d2736d819d5021de4c20a
#1 transferring dockerfile: 1.58kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:d76781e46582c3b8461b64e7ec58b26ebd3560dbc2f1aeeb628792771d7209ff
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:4a2208b6298ccbc9cc70222a6c15f7a129c1cd5746c6c5a1dd63bd1aa06272a5
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 DONE 4.7s

#5 [ 1/12] FROM docker.io/l

## Start the docker container

Run this command in a separate shell, because it needs to run in the background. It will be killed with ^C.

```
docker run -p 5001:8000 -e DISABLE_NGINX=true -e GUNICORN_CMD_ARGS="--bind=0.0.0.0" my-image-name:latest
```

## Try to get a prediction

In [12]:
import requests
import json
import pandas as pd

dataset = pd.read_csv('data/train_bike.csv', header=0,  sep=',',
                      parse_dates=['datetime'], index_col='datetime').drop(
    columns=['casual', 'registered']
)
test_year = 2011
test_month = 3
test: pd.DataFrame = dataset.loc[
    lambda df: (df.index.year == test_year) & (df.index.month == test_month)
]
def time_features(input_df):
    return input_df.assign(
        month = lambda df: df.index.month,
        day = lambda df: df.index.day,
        hour = lambda df: df.index.hour,
    )
test.pipe(time_features).drop(columns='count').head().to_json(orient='split', index=False)

predictions = requests.post(
    'http://127.0.0.1:5001/invocations',
    json=json.loads(
        test.pipe(time_features)
        .drop(columns='count')
        .to_json(orient='split', index=False)
    )
).json()

from sklearn.metrics import r2_score

r2_score(test.pipe(time_features)['count'].values, predictions)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))